In [1]:
from http.server import SimpleHTTPRequestHandler , HTTPServer
from urllib import  request , error 
import pandas as pd 
from pycaret.clustering import *
import urllib.parse
import sys 

badwords = ['sleep','drop','uid','select','waitfor','delay','system','union','order by','group by']
def ExtractFeatures(path):
    path = urllib.parse.unquote(path)
    badwords_count = 0
    single_q = path.count("'")
    double_q = path.count("\"")
    dashes = path.count("--")
    braces = path.count("(")
    spaces =path.count(" ")
    for word in badwords:
        badwords_count += path.count(word)
    lst = [single_q,double_q,dashes,braces,spaces,badwords_count]
    print(lst)
    return pd.DataFrame([lst] , columns =  ['single_q','double_q','dashes','braces','spaces','badwords'])

C:\Users\lenovo\anaconda3\envs\Pycaret\lib\site-packages\pycaret\loggers\mlflow_logger.py:14: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  import mlflow


In [2]:
#training our model
http = pd.read_csv(r'C:\Users\lenovo\Documents\PFE\legit\all.csv')
clu1 = setup(data = http, normalize = True,numeric_features = ['single_q','double_q','dashes','braces','spaces','badwords',], ignore_features = ['method','path','body','class'])
kmeans = create_model('kmeans',num_clusters = 2)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.8491,6107.5062,0.9194,0,0,0


In [ ]:
class SimpleHTTPProxy(SimpleHTTPRequestHandler):
    proxy_routes = {}
    @classmethod
    def set_routes(cls, proxy_routes):
        cls.proxy_routes = proxy_routes
    
    def do_GET(self):
        #print (self.proxy_routes)
        #print (self.path)
        parts = self.path.split('/')
        print (parts)
        live_data = ExtractFeatures(parts[3])
        result = predict_model(kmeans , data = live_data)
        print(result['Cluster'][0])
        if result['Cluster'][0] == "Cluster 1":
            print('Intrusion Detected !')
        #print (parts,Len(parts),part[1])
        if len(parts) >= 2:
            #url = self.proxy_routes[parts[2]] + '/'.join(parts[2:])
            #print(url)
            self.proxy_request('http://'+parts[2]+'/')
        else:
            super().do_GET()
        
    def proxy_request(self,url):
        #print('proxy req')
        #exit()
        try:
            response = request.urlopen(url)
        except error.HTTPError as e:
            #print('error')
            self.send_response_only(e.code)
            self.end_headers()
            return
        self.send_response_only(response.status)
        for name, value in response.headers.items():
            self.send_header(name, value)
        self.end_headers()
        self.copyfile(response, self.wfile)
SimpleHTTPProxy.set_routes({'proxy_route':'http://demo.testfire.net/'})
with HTTPServer(('127.0.0.1',8090), SimpleHTTPProxy) as httpd :
    host , port = httpd.socket.getsockname()
    print(f'Listening on http://{host}:{port}')
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("\nKeyboard interrupt received , exiting .")

Listening on http://127.0.0.1:8090


127.0.0.1 - - [19/Feb/2023 12:43:08] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [19/Feb/2023 12:43:08] "CONNECT demo.testfire.net:443 HTTP/1.1" 501 -
127.0.0.1 - - [19/Feb/2023 12:43:32] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [19/Feb/2023 12:43:32] "CONNECT demo.testfire.net:443 HTTP/1.1" 501 -
127.0.0.1 - - [19/Feb/2023 12:43:39] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [19/Feb/2023 12:43:39] "CONNECT demo.testfire.net:443 HTTP/1.1" 501 -


['http:', '', 'detectportal.firefox.com', 'success.txt?ipv4']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'success.txt?ipv6']
[0, 0, 0, 0, 0, 0]
Cluster 0


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 57091)
Traceback (most recent call last):
  File "C:\Users\lenovo\anaconda3\envs\Pycaret\lib\socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\lenovo\anaconda3\envs\Pycaret\lib\socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\lenovo\anaconda3\envs\Pycaret\lib\socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\lenovo\anaconda3\envs\Pycaret\lib\socketserver.py", line 724, in __init__
    self.handle()
  File "C:\Users\lenovo\anaconda3\envs\Pycaret\lib\http\server.py", line 418, in handle
    self.handle_one_request()
  File "C:\Users\lenovo\anaconda3\envs\Pycaret\lib\http\server.py", line 406, in handle_one_request
    method()
  File "<ipython-input-3-15e3767df973>", line 21, 

['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'success.txt?ipv4']
[0, 0, 0, 0, 0, 0]


...

127.0.0.1 - - [19/Feb/2023 13:40:05] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [19/Feb/2023 13:40:05] "CONNECT contile.services.mozilla.com:443 HTTP/1.1" 501 -


['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detect

['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detect

127.0.0.1 - - [19/Feb/2023 13:48:51] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [19/Feb/2023 13:48:51] "CONNECT safebrowsing.googleapis.com:443 HTTP/1.1" 501 -


['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detectportal.firefox.com', 'canonical.html']
[0, 0, 0, 0, 0, 0]
Cluster 0
['http:', '', 'detect